---
comments: true
layout: post
title: CPT Project Documentation
description: documentation. will have frqs later
type: tangibles
courses: { compsci: {week: 29} }
---

# Python Model code and SQLite Database

- From VSCode using SQLite3 Editor, show your unique collection/table in database, display rows and columns in the table of the SQLite database.

Users:
[![Screenshot-2024-04-16-at-9-29-27-PM.png](https://i.postimg.cc/QNm2PQKB/Screenshot-2024-04-16-at-9-29-27-PM.png)](https://postimg.cc/HjJv872H)

Designs:
[![Screenshot-2024-04-17-at-12-58-15-PM.png](https://i.postimg.cc/Kz9WFqxG/Screenshot-2024-04-17-at-12-58-15-PM.png)](https://postimg.cc/mzHjySPJ)

- From VSCode model, show your unique code that was created to initialize table and create test data.

Users:
[![Screenshot-2024-04-16-at-9-32-35-PM.png](https://i.postimg.cc/VvJ2nTf3/Screenshot-2024-04-16-at-9-32-35-PM.png)](https://postimg.cc/bSj6jm2T)

Designs is initialized as empty.

Columns will 

# Lists and Dictionaries

- In VSCode using Debugger, show a list as extracted from database as Python objects.

[![Screenshot-2024-04-17-at-9-07-21-PM.png](https://i.postimg.cc/GtBQWQLZ/Screenshot-2024-04-17-at-9-07-21-PM.png)](https://postimg.cc/N2cmTRr4)

The JS sends a GET request to the backend and collects all public designs into a list design_return, with each design as a python object (shown below).


- In VSCode use Debugger and list, show examples of dictionaries, show Keys/Values using debugger.

[![Screenshot-2024-04-17-at-9-19-16-PM.png](https://i.postimg.cc/Z5SdhSJT/Screenshot-2024-04-17-at-9-19-16-PM.png)](https://postimg.cc/ZCwRrX9g)

In each button object, the dictionary contains keys for "Name", "Content", "Description", and owner id, along with number of likes and dislikes and the type (public/private). The search function makes a GET request and the backend handles this by returning all the public designs and sending them off to the frontend to process there based on search term.

# Python API code, Postman/JSON

- In VSCode, show Python API code definition for request and response using GET, POST, UPDATE methods. Discuss algorithmic condition used to direct request to appropriate Python method based on request method.

[![Screenshot-2024-04-16-at-10-03-30-PM.png](https://i.postimg.cc/W3Bs83T7/Screenshot-2024-04-16-at-10-03-30-PM.png)](https://postimg.cc/MMYkKzcc)

Above are the GET and PUT methods for the SEARCH CRUD class. Either one is selected based on "if" the user selects the Public or Private designs button on the website page. 

[![Screenshot-2024-04-16-at-10-07-46-PM.png](https://i.postimg.cc/qvGmkZFc/Screenshot-2024-04-16-at-10-07-46-PM.png)](https://postimg.cc/K1R7fQx4)

Below is the frontend JS code to handle user requests with if statements:

[![Screenshot-2024-04-16-at-10-08-42-PM.png](https://i.postimg.cc/RhMhzhJL/Screenshot-2024-04-16-at-10-08-42-PM.png)](https://postimg.cc/8J0NR1H7)

This is the POST method for the USER CRUD class. 

[![Screenshot-2024-04-16-at-10-10-32-PM.png](https://i.postimg.cc/Xvd9jZqR/Screenshot-2024-04-16-at-10-10-32-PM.png)](https://postimg.cc/2qj1TS4G)

The POST method takes the JSON data from the user and checks of any of the fields/parameters are invalid with if statements. It then creates the User object for a new user and inserts the data into the SQLite database. 

The UPDATE method is embedded within the PUT method for the same class. Requests are handled similarly to the above, and @token_required only allows requests to be made if the user is authenticated and a token is present.

[![Screenshot-2024-04-16-at-10-31-03-PM.png](https://i.postimg.cc/k5ms6VDd/Screenshot-2024-04-16-at-10-31-03-PM.png)](https://postimg.cc/dh5rgV8W)

- In VSCode, show algorithmic conditions used to validate data on a POST condition.

[![Screenshot-2024-04-16-at-10-34-27-PM.png](https://i.postimg.cc/x1hqj7jk/Screenshot-2024-04-16-at-10-34-27-PM.png)](https://postimg.cc/SnLmDgMq)

Also shown above in the POST method code. Checks if each field is at least 2 characters, and if not it returns a corresponding message and exits the POST method function.

- In Postman, show URL request and Body requirements for GET, POST, and UPDATE methods.

POST: (create user)

[![Screenshot-2024-04-16-at-10-50-24-PM.png](https://i.postimg.cc/zvGgMcFx/Screenshot-2024-04-16-at-10-50-24-PM.png)](https://postimg.cc/5YDyQpGz)

uid, password, name

GET: (search)

[![Screenshot-2024-04-16-at-10-56-30-PM.png](https://i.postimg.cc/BZ8Cp8tH/Screenshot-2024-04-16-at-10-56-30-PM.png)](https://postimg.cc/3WTpJxXx)

No body is required for the GET in the search.

UPDATE: (aka PUT, for search)

[![Screenshot-2024-04-16-at-11-05-18-PM.png](https://i.postimg.cc/wB1hbbgq/Screenshot-2024-04-16-at-11-05-18-PM.png)](https://postimg.cc/KKyk1f1w)

- In Postman, show the JSON response data for 200 success conditions on GET, POST, and UPDATE methods.

POST success:

[![Screenshot-2024-04-16-at-11-02-05-PM.png](https://i.postimg.cc/65KqbSYw/Screenshot-2024-04-16-at-11-02-05-PM.png)](https://postimg.cc/RqPvNgkb)

UPDATE/PUT success:

[![Screenshot-2024-04-16-at-11-04-18-PM.png](https://i.postimg.cc/C1yFpzTZ/Screenshot-2024-04-16-at-11-04-18-PM.png)](https://postimg.cc/CZNyjMKY)

GET success:

[![Screenshot-2024-04-17-at-10-28-06-AM.png](https://i.postimg.cc/nrmTKbhR/Screenshot-2024-04-17-at-10-28-06-AM.png)](https://postimg.cc/Pvtmh7ZY)

- In Postman, show the JSON response for error for 400 when missing body on a POST request.

[![Screenshot-2024-04-17-at-10-30-07-AM.png](https://i.postimg.cc/dQ9h1N6w/Screenshot-2024-04-17-at-10-30-07-AM.png)](https://postimg.cc/hhfDZrN5)

Here, the only data provided was {}.

- In Postman, show the JSON response for error for 404 when providing an unknown user ID to a UPDATE request.

Our user API code automatically retrieves user ID from the JWT token from login by decoding it, meaning that no uid needs to be provided with the UPDATE/PUT request. This ensures that users can only update their own user profile, and also means that there is no possibility for a 404 error response (no uid = no unknown uid).

Extra: 401 error when searching for private designs

[![Screenshot-2024-04-16-at-11-00-25-PM.png](https://i.postimg.cc/ryCYx0Hj/Screenshot-2024-04-16-at-11-00-25-PM.png)](https://postimg.cc/Jt0qWnMH)

@token_required takes effect here because the user is not authenticated and thus no jwt token is present.

# Frontend

- Blog JavaScript API fetch code and formatting code to display JSON.

Below are both search functions (getPublic and getPrivate), which are GET and PUT requests respectively. If the data is returned successfully, then the last function displayDataInTable is called with the argument containing the designs. displayDataInTable then creates the HTML table to display all relevant search results in the page. It loops through all the returned designs and checks if they contain the user's search term within the name or the content, adding new rows accordingly.

In [ ]:
function getPublic() { // splitting up public and private cases
            fetch('http://127.0.0.1:8086/api/users/search')
                .then(response => {
                    if (!response.ok) {
                        throw new Error('Network response was not ok');
                    }
                    return response.json();
                })
                .then(data => {
                    console.log(data); // Handle the data returned from the server
                    displayDataInTable(data.Designs);
                })
                .catch(error => {
                    console.error('There was a problem with the fetch operation:', error);
                });
}
        
function getPrivate() {
            // Making the PUT request (private)
            const authOptions = {
                mode: 'cors', // no-cors, *cors, same-origin
                credentials: 'include', // include, same-origin, omit
                headers: {
                    'Content-Type': 'application/json',
                },
                method: 'PUT', // Override the method property
            };
            fetch('http://127.0.0.1:8086/api/users/search', authOptions)
                .then(response => {
                    if (!response.ok) {
                        throw new Error('Network response was not ok');
                    }
                    return response.json();
                })
                .then(data => {
                    console.log(data); // Handle the data returned from the server
                    displayDataInTable(data.Designs);
                })
                .catch(error => {
                    console.error('There was a problem with the fetch operation:', error);
                });
        }
function displayDataInTable(data) {
            var tableContainer = document.getElementById('tableContainer');
            var tableHTML = '<table id="table">'; // initialize the table
            const searchTerm = getSearchTerm(); // item in search field
            tableHTML += '<tr><th>Name</th><th>Content</th><th>Description</th><th>Likes</th><th>Dislikes</th><th>Type</th></tr>';
            data.forEach(function(item) {
                // Check if searchTerm is a substring of Name or Content
                if (searchTerm && (item.Name.includes(searchTerm) || (item.Content && item.Content.includes(searchTerm)))) {
                    tableHTML += '<tr>';
                    tableHTML += '<td class="nr">' + item.Name + '</td>';
                    tableHTML += '<td>' + (item.Content || '') + '</td>';
                    tableHTML += '<td>' + item.Description + '</td>';
                    tableHTML += '<td><button onclick="toggleLike(this, this.closest(`tr`).querySelector(`.nr`).textContent)" data-item-id="' + item.id + '">Like</button><span class="likesCount">' + item.Likes + '</span></td>';
                    tableHTML += '<td><button onclick="toggleDislike(this, this.closest(`tr`).querySelector(`.nr`).textContent)" data-item-id="' + item.id + '">Dislike</button><span class="dislikesCount">' + item.Dislikes + '</span></td>';
                    tableHTML += '<td>' + item.Type + '</td>';
                    tableHTML += '</tr>';
                }
            });
            tableHTML += '</table>';
            tableContainer.innerHTML = tableHTML;
        }

- In Chrome inspect, show response of JSON objects from fetch of GET, POST, and UPDATE methods.

GET: 

[![Screenshot-2024-04-17-at-11-44-37-AM.png](https://i.postimg.cc/y6hfvRhv/Screenshot-2024-04-17-at-11-44-37-AM.png)](https://postimg.cc/t7TtgYMx)

POST and UPDATE/PUT (for users) do not return JSON objects. They redirect the browser window to the homepage of the site if successful. 


- In the Chrome browser, show a demo (GET) of obtaining an Array of JSON objects that are formatted into the browsers screen.

[![Screenshot-2024-04-17-at-11-44-37-AM.png](https://i.postimg.cc/y6hfvRhv/Screenshot-2024-04-17-at-11-44-37-AM.png)](https://postimg.cc/t7TtgYMx)

This GET method obtains the list of public designs when the user searches. It returns the array of JSON objects and formats it into the HTML table.

- In JavaScript code, describe fetch and method that obtained the Array of JSON objects.

In [ ]:
fetch('http://127.0.0.1:8086/api/users/search')
    .then(response => {
        if (!response.ok) {
            throw new Error('Network response was not ok');
        }
        return response.json();
    })
    .then(data => {
        console.log(data); // Handle the data returned from the server
        displayDataInTable(data.Designs);
    })
    .catch(error => {
        console.error('There was a problem with the fetch operation:', error);
    });

The code fetches to the url on the first line, with a GET method (default) the code handles the request from the backend by verifying the data first and then displays the code in the table.

- In JavaScript code, show code that performs iteration and formatting of data into HTML.

In [ ]:
function displayDataInTable(data) {
    var tableContainer = document.getElementById('tableContainer');
    var tableHTML = '<table id="table">'; // initialize the table
    const searchTerm = getSearchTerm(); // item in search field
    tableHTML += '<tr><th>Name</th><th>Content</th><th>Description</th><th>Likes</th><th>Dislikes</th><th>Type</th></tr>';
    data.forEach(function(item) {
        // Check if searchTerm is a substring of Name or Content
        if (searchTerm && (item.Name.includes(searchTerm) || (item.Content && item.Content.includes(searchTerm)))) {
            tableHTML += '<tr>';
            tableHTML += '<td class="nr">' + item.Name + '</td>';
            tableHTML += '<td>' + (item.Content || '') + '</td>';
            tableHTML += '<td>' + item.Description + '</td>';
            tableHTML += '<td><button onclick="toggleLike(this, this.closest(`tr`).querySelector(`.nr`).textContent)" data-item-id="' + item.id + '">Like</button><span class="likesCount">' + item.Likes + '</span></td>';
            tableHTML += '<td><button onclick="toggleDislike(this, this.closest(`tr`).querySelector(`.nr`).textContent)" data-item-id="' + item.id + '">Dislike</button><span class="dislikesCount">' + item.Dislikes + '</span></td>';
            tableHTML += '<td>' + item.Type + '</td>';
            tableHTML += '</tr>';
        } // build new table row with cells containing the details of the design
    });
    tableHTML += '</table>';
    tableContainer.innerHTML = tableHTML;
}

The code gets the container id of the table and initializes the HTML string with a header row. Then iterates through each element of the designs data, and checks if the content or the name contains the search term from the user. If the design is valid (what the user searched for), then it appends it to the HTML string.

- In the Chrome browser, show a demo (POST or UPDATE) gathering and sending input and receiving a response that show update. Repeat this demo showing both success and failure.

UPDATE/PUT request success:
[![Screenshot-2024-04-17-at-12-16-56-PM.png](https://i.postimg.cc/3JMJXdC8/Screenshot-2024-04-17-at-12-16-56-PM.png)](https://postimg.cc/pyZHxV64)

POST request failure:
[![Screenshot-2024-04-17-at-12-18-07-PM.png](https://i.postimg.cc/Qd42qzGQ/Screenshot-2024-04-17-at-12-18-07-PM.png)](https://postimg.cc/WdZYNX4z)

- In JavaScript code, show and describe code that handles failure. Describe how the code shows failure to the user in the Chrome Browser screen.

In [ ]:
function closePopup() {
        const name1 = document.getElementById("nameInput").value;
        const publicInput1 = document.getElementById("publicInput").value;
        const description = document.getElementById("description").value;
        const body = {
            name: name1,
            content: "",
            type: publicInput1,
            description: description
        }
        console.log(name1)
        console.log(description)
        if (name1.trim() !== "") {
            const authOptions = {
                mode: 'cors', // no-cors, *cors, same-origin
                credentials: 'include', // include, same-origin, omit
                headers: {
                    'Content-Type': 'application/json',
                },
                method: 'POST', // Override the method property
                cache: 'no-cache', // Set the cache property
                body: JSON.stringify(body)
            };
            console.log(body)
            fetch(url, authOptions)
            .then(response => {
                // handle error response from Web API
                if (!response.ok) {
                    window.location.href = "http://127.0.0.1:4100/CPT/login";
                }
                document.getElementById("popup").style.display = "none";
            })
            // catch fetch errors (ie ACCESS to server blocked)
            .catch(err => {
                console.log(body);
                console.error(err);
            });
        } else {
            alert("Please enter a design name!");
        }
    }

If the user is not logged in, it redirects the user to the login page. Otherwise, if the user has not inputted a design name, an alert pops up and the design is not created (no fetch is made).

- In JavaScript code, show and describe code that handles success. Describe how code shows success to the user in the Chrome Browser screen.

In [ ]:
function saveButton(name) { 
    const url ='http://127.0.0.1:8086/api/users/design';
    const body = {
        name: name,
        content: buttonCode.value,
        type: "",
        description: description
    }
    const authOptions = {
        mode: 'cors', // no-cors, *cors, same-origin
        credentials: 'include', // include, same-origin, omit
        headers: {
            'Content-Type': 'application/json',
        },
        method: 'PUT', // Override the method property
        cache: 'no-cache', // Set the cache property
        body: JSON.stringify(body)
    };
    fetch(url, authOptions)
    .then(response => {
        // handle error response from Web API
        if (!response.ok) {
            const errorMsg = 'Save error: ' + response.status;
            console.log(errorMsg);
            return;
        } else {
            alert("Design Saved")
        }
    })
    // catch fetch errors (ie ACCESS to server blocked)
    .catch(err => {
        console.log(body);
        console.error(err);
    });
}

The code checks if save (the PUT/UPDATE request) is successful, and alerts the user with a popup based on the result.

# /model/concussion.py - ML Linear Regression
This is the model for the concussions recovery time predict machine learning model. I used linear regression on generated data in a CSV file to predict the "healtime" variable, then output to the user.

In [50]:
# import all necessary packages for machine learning
# numpy to handle the returned result and convert into usable list format
import numpy as np
# need to be able to split up data into training and testing sets
from sklearn.model_selection import train_test_split
# training with linear regression model as data is continuous, not discrete
from sklearn.linear_model import LinearRegression
# python pandas package
# need pandas for dataframe and easy data manipulation
import pandas as pd

# Define the concussion_regression global variable
concussion_regression = None

# define the ConcussionRegression class
# we are using OOP here
class ConcussionRegression:
    # below are the separate functions to handle various events in the prediction
    # initConcussion will take the generated sample data from the CSV file
    # then it will clean data by removing all negative values
    def initConcussion(self):
        # load in the csv data
        concussion_data = pd.read_csv('concussion_recovery_data.csv')
        '''
        Categories in the CSV file: sex,age,ht,wt,smoke,alcohol,sleephrs,exercisehrs,hitbox,healtime
        sex is boolean, 1=male 0=female
        ht = height (cm)
        wt = weight (kg)
        smoke/alcohol are boolean, 1=yes 0=no
        sleephrs are per night
        exercisehrs are per week
        hitbox = size of object hit, in kg
        healtime is in days
        '''
        global cd
        # now variable cd has all the data
        cd = concussion_data
        categories = ['age', 'ht', 'wt', 'sleephrs', 'exercisehrs', 'hitbox', 'healtime']
        # clean up the data
        # for non-boolean categories, drop all negative values
        for cat in categories:
            cd.drop(cd[cd[cat] < 0].index, inplace=True)
    
    # runLinearRegression trains the model according to linear regression
    # the model will be trained to predict healtime based on the other categories
    def runLinearRegression(self):
        # making a linear regression model
        X = cd.drop('healtime', axis=1) # all categories except healtime (result)
        y = cd['healtime'] # healtime (the result)
        # split up data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        global lr
        # train the model
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        
def predict(data):
    new_case = data.copy() # copy in the data from frontend
    '''
    format of the input:
    new_case = {
            'name': case.get('name'),
            'sex': case.get('sex'),
            'ht': case.get('ht'),
            'wt': case.get('wt'),
            'smoke': case.get('smoke'),
            'alcohol': case.get('alcohol'),
            'sleephrs': case.get('sleephrs'),
            'exercisehrs': case.get('exercisehrs'),
            'hitbox': case.get('hitbox'),
        }
    '''
    # convert frontend data to usable format
    # sex converted to boolean and name doesn't matter
    new_case['sex'] = new_case['sex'].apply(lambda x: 1 if x == 'male' else 0)
    new_case.drop(['name'], axis=1, inplace=True)
    # predict time to heal
    # need to convert to list since numpy arrays are not directly serializable to JSON
    healtime = np.squeeze(lr.predict(new_case)).tolist() # use numpy and .tolist()
    # print(healtime)
    return healtime

def initConcussion():
    # initiate the concussion regression model
    # then run linear regression
    global concussion_regression
    concussion_regression = ConcussionRegression()
    concussion_regression.initConcussion()
    concussion_regression.runLinearRegression()
    
# test the model
if __name__ == "__main__":
    # initialize concussion model
    initConcussion()
    # test data on partner
    patient_data = pd.DataFrame({
        'name': ['Andrew Kim'],
        'sex': ['male'],
        'age': [16],
        'ht': [180],
        'wt': [73],
        'smoke': [0],
        'alcohol': [1],
        'sleephrs': [8.5],
        'exercisehrs': [5.5],
        'hitbox': [0.28],
    })
    print(predict(patient_data))

10.526551391730155
